# Описание проекта
Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

### Описание данных

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

### План работы

### [Bert](#b)
1. [Подготовка данных](#b_1)
2. [Обучение модели](#b_2)

### [tf-idf](#t)

1. [Подготовка данных](#t_1)
2. [Обучение модели](#t_2)

### [Выводы](#c)

### Импорты библиотек

In [2]:
import os
from urllib.request import urlretrieve

import pandas as pd
import numpy as np
from tqdm import notebook

# NLP
import torch
import transformers
import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer

# ML
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression, SGDClassifier
from catboost import CatBoostClassifier

### Функции

In [3]:
def get_file(file_name, url):
    if not os.path.exists(file_name):
        print('Файл не найден и будет загружен из сети')
        file_name, headers = urlretrieve(url)
    return pd.read_csv(file_name)

### Загрузка данных

In [4]:
file_name = 'toxic_comments.csv'
url = '/datasets/toxic_comments.csv'

df = get_file(file_name, url)
print('Размер датасета:', df.shape)
df.head()

Файл не найден и будет загружен из сети
Размер датасета: (159571, 2)


,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


<a id='b'></a>
# BERT

<a id='b_1'></a>
### 1. Подготовка

1. Сразу выделим тестовую выборку и сбалансируем трейн с помощью downsampling

In [5]:
train, test = train_test_split(df, test_size=0.2, random_state=333)

train_0 = train[train['toxic'] == 0].sample(10000, random_state=333)
train_1 = train[train['toxic'] == 1].sample(10000, random_state=333)
train = shuffle(pd.concat([train_0]+[train_1]))

test = test.sample(10000, random_state=333)

print('Размер трейна', train.shape)
print('Размер теста', test.shape)

Размер трейна (20000, 2)
Размер теста (10000, 2)


2. Выполняем токенизацию каждого текста, то есть разбиваем на слова 
(Алгоритм лемматизации и очистки текста уже заложен в модели Bert)
4. На выходе у каждого исходного текста образуется свой список токенов.

In [6]:
%%time

# tockenize
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-cased')
tokenized = train['text'].apply(
    lambda x: tokenizer.encode(x[:512], add_special_tokens=True))

# count max qty of tockens
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
# add 0 to strokes which lenght less than max
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])
# make a mask to highlight important tockens
attention_mask = np.where(padded != 0, 1, 0)
padded.shape


CPU times: user 19.8 s, sys: 132 ms, total: 19.9 s
Wall time: 21.5 s


(20000, 486)

7. Затем токены передаем модели, которая переводит их в векторные представления. Для этого модель обращается к составленному заранее словарю токенов. На выходе для каждого текста образуются векторы заданной длины.

In [7]:
%%time

device = torch.device('cuda')

bert_model = transformers.BertModel.from_pretrained('bert-base-uncased')
bert_model.to(device)

batch_size = 50
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.cuda.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.cuda.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = bert_model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())

train_x = np.concatenate(embeddings)
train_y = train['toxic']

AssertionError: 
Found no NVIDIA driver on your system. Please check that you
have an NVIDIA GPU and installed a driver from
http://www.nvidia.com/Download/index.aspx

Проделываем все тоже самое с тестовой выборкой

In [7]:
%%time
# tockenize
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-cased')
tokenized = test['text'].apply(
    lambda x: tokenizer.encode(x[:128], add_special_tokens=True))

# count max qty of tockens
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
# add 0 to strokes which lenght less than max
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])
# make a mask to highlight important tockens
attention_mask = np.where(padded != 0, 1, 0)
padded.shape

Wall time: 4.25 s


(10000, 127)

In [8]:
%%time
device = torch.device('cuda')

bert_model = transformers.BertModel.from_pretrained('bert-base-uncased')
bert_model.to(device)

batch_size = 50
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.cuda.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.cuda.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = bert_model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())

test_x = np.concatenate(embeddings)
test_y = test['toxic']


Wall time: 27.6 s


<a id='b_2'></a>
### 2. Обучение

8. На финальном этапе модели передаем признаки (векторы). И она прогнозирует эмоциональную окраску текста — 0 («отрицательная») или 1 («положительная»).

In [9]:
lr_model = LogisticRegression(random_state=333, class_weight='balanced', solver='liblinear')
lr_model.fit(train_x, train_y)

train_pred = lr_model.predict(train_x)
test_pred = lr_model.predict(test_x)

print('f1 на трейне:', f1_score(train_y, train_pred))
print('f1 на тесте:', f1_score(test_y, test_pred))

f1 на трейне: 0.7639659825838977
f1 на тесте: 0.3242313480971834


In [10]:
sgd_model = SGDClassifier(random_state=333, class_weight='balanced')
sgd_model.fit(train_x, train_y)

train_pred = sgd_model.predict(train_x)
test_pred = sgd_model.predict(test_x)

print('f1 на трейне:', f1_score(train_y, train_pred))
print('f1 на тесте:', f1_score(test_y, test_pred))

f1 на трейне: 0.7738889645467338
f1 на тесте: 0.2768734891216761


На тесте очень слабый показатель, возможно ситуация будет лучше если увеличить выборку и убрать downsampling

************************************

<a id='t'></a>
# tf-idf

<a id='t_1'></a>
### 1. Подготовка

In [8]:
df = df.sample(100000).reset_index(drop=True)

1. Слова лемматризируем
2. Текст очищаем от стоп-слов и ненужных символов

In [9]:
def stemmatize(text):
    stemmer = nltk.stem.snowball.SnowballStemmer('english')
    
    words = text.split()
    stemm_list = []
    for word in words:
        stemm_list.append(stemmer.stem(word))
        
    return ' '.join(stemm_list)


def clear_text(text):
    processed_text = re.sub(r'[^a-zA-Z]', ' ', text)
    processed_text = ' '.join(processed_text.split())
    
    return processed_text

In [10]:
%%time
df['stem'] = df['text'].apply(lambda x: stemmatize(clear_text(x)))

CPU times: user 1min 34s, sys: 166 ms, total: 1min 34s
Wall time: 1min 34s


Выделим тестовую выборку

In [11]:
train, test = train_test_split(df, test_size=0.2, random_state=333)

train_x = train.drop('toxic', axis=1)
train_y = train['toxic']

test_x = test.drop('toxic', axis=1)
test_y = test['toxic']

print('Размер трейна', train_x.shape)
print('Размер теста', test_x.shape)

Размер трейна (80000, 2)
Размер теста (20000, 2)


3. Готовим векторы признаков

In [12]:
%%time
count_tfidf = TfidfVectorizer(stop_words='english')

tf_idf = count_tfidf.fit_transform(np.array(train_x['stem']))
train_x = pd.DataFrame(tf_idf.toarray())

tf_idf = count_tfidf.transform(np.array(test_x['stem']))
test_x = pd.DataFrame(tf_idf.toarray())

print('Размер трейна', train_x.shape)
print('Размер теста', test_x.shape)

MemoryError: Unable to allocate 51.7 GiB for an array with shape (80000, 86713) and data type float64

In [17]:
%%time
from nltk.corpus import stopwords
count_tfidf = TfidfVectorizer(stop_words=stopwords.words('english'))

tf_idf_train = count_tfidf.fit_transform(train_x['stem'])
#train_x = pd.DataFrame(tf_idf.toarray())

tf_idf_test = count_tfidf.transform(test_x['stem'])
#test_x = pd.DataFrame(tf_idf.toarray())

#print('Размер трейна', train_x.shape)
#print('Размер теста', test_x.shape)

CPU times: user 6.22 s, sys: 20 ms, total: 6.24 s
Wall time: 6.26 s


In [18]:
%%time
lr_model = LogisticRegression(random_state=333, class_weight='balanced', solver='liblinear')
lr_model.fit(tf_idf_train, train_y)

train_pred = lr_model.predict(tf_idf_train)
test_pred = lr_model.predict(tf_idf_test)

print('f1 на трейне:', f1_score(train_y, train_pred))
print('f1 на тесте:', f1_score(test_y, test_pred))

f1 на трейне: 0.8330152671755725
f1 на тесте: 0.7400043224551545
CPU times: user 3.02 s, sys: 32.7 ms, total: 3.06 s
Wall time: 3.06 s


<a id='t_2'></a>
### 2. Обучение

LogisticRegression

In [9]:
%%time
lr_model = LogisticRegression(random_state=333, class_weight='balanced', solver='liblinear')
lr_model.fit(train_x, train_y)

train_pred = lr_model.predict(train_x)
test_pred = lr_model.predict(test_x)

print('f1 на трейне:', f1_score(train_y, train_pred))
print('f1 на тесте:', f1_score(test_y, test_pred))

f1 на трейне: 0.8406642967050459
f1 на тесте: 0.7584562012142239
CPU times: user 1min 54s, sys: 2min 19s, total: 4min 13s
Wall time: 4min 12s


In [13]:
%%time
cat_model = CatBoostClassifier(random_state=333)
cat_model.fit(train_x, train_y, verbose=100)

train_pred = cat_model.predict(train_x)
test_pred = cat_model.predict(test_x)

print('f1 на трейне:', f1_score(train_y, train_pred))
print('f1 на тесте:', f1_score(test_y, test_pred))

Learning rate set to 0.06692
0:	learn: 0.6245754	total: 1.09s	remaining: 18m 13s
1:	learn: 0.5665365	total: 1.54s	remaining: 12m 51s
2:	learn: 0.5165582	total: 2s	remaining: 11m 4s
3:	learn: 0.4734110	total: 2.52s	remaining: 10m 26s
4:	learn: 0.4381480	total: 3s	remaining: 9m 56s
5:	learn: 0.4072238	total: 3.44s	remaining: 9m 30s
6:	learn: 0.3800057	total: 3.93s	remaining: 9m 17s
7:	learn: 0.3571030	total: 4.39s	remaining: 9m 4s
8:	learn: 0.3377110	total: 4.84s	remaining: 8m 52s
9:	learn: 0.3215158	total: 5.29s	remaining: 8m 43s
10:	learn: 0.3077345	total: 5.76s	remaining: 8m 37s
11:	learn: 0.2958274	total: 6.22s	remaining: 8m 32s
12:	learn: 0.2854239	total: 6.7s	remaining: 8m 28s
13:	learn: 0.2776176	total: 7.22s	remaining: 8m 28s
14:	learn: 0.2701312	total: 7.71s	remaining: 8m 26s
15:	learn: 0.2634592	total: 8.15s	remaining: 8m 21s
16:	learn: 0.2574907	total: 8.6s	remaining: 8m 17s
17:	learn: 0.2519080	total: 9.05s	remaining: 8m 13s
18:	learn: 0.2466453	total: 9.5s	remaining: 8m 10s


In [14]:
%%time
sgd_model = SGDClassifier(random_state=333, class_weight='balanced')
sgd_model.fit(train_x, train_y)

train_pred = sgd_model.predict(train_x)
test_pred = sgd_model.predict(test_x)

print('f1 на трейне:', f1_score(train_y, train_pred))
print('f1 на тесте:', f1_score(test_y, test_pred))

f1 на трейне: 0.8111812638281799
f1 на тесте: 0.753731343283582
CPU times: user 10min 28s, sys: 9min 10s, total: 19min 38s
Wall time: 25min 59s


<a id='c'></a>
# Выводы

Балансировку классов провели с помощью внутринних механизмов моделей.

Лучший результат f1 0.758 на тесте показала логистическая регрессия поверх tf-idf. Возможно лог регрессия поверх Bertа показала бы результат лучше при большей выборке.